## Data Analysis Pipeline Steps
1. Import & concatenate data:
- Write a function that takes a folder of data files as an argument and concatenates all the .csv files in that folder into a single pandas dataframe.
    - *hint*: use the packages "glob" and "pandas"
    - *hint2*: you need to skip the first three rows of each .csv file for this to work.
2. Average response score for each object.
- Get a summary of the average score for each object shown.
    - *hint*: There are a couple of ways to do this, you can use the *.groupby()* method in pandas to group items in the 'object_shown' column, or you can create a pivot table.
- Check whether any objects have average response scores lower than 6.
3. Find the images we want to use.
- Group by images (in the 'stim1' column) and get the average score for each stim1. 
    - *hint* use the .groupby() and .mean() methods
- For each object, find the two highest-scored images that have the same score. 
    - If no two objects have the same score, list the two highest scores for each object. 
    - If more than two objects have matching highest-scores, list all of them.

In [1]:
import pandas as pd
import numpy as np
import glob